# pandas - Data Cleaning and Preparation

In [178]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

## Handling Missing Data

In [179]:
string_data = Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

In [180]:
string_data.isnull()

In [181]:
string_data[0] = None # None is not null or NaN
string_data.isnull()

### Filtering Out Missing Data

In [182]:
from numpy import nan as NA
data = Series([1, NA, 3.5, NA, 7])
data.dropna() # drop missing values

In [183]:
data[data.notnull()] # drop missing values

In [184]:
data = DataFrame([[1., 6.5, 3.], [1., NA, NA],
                 [NA, NA, NA], [NA, 6.5, 3.]])
data

In [185]:
cleaned = data.dropna()
cleaned # if one of the value is NaN then the whole row gets dropped

In [186]:
data.dropna(how='all') # drop the rows that are all NA

In [187]:
data[4] = NA
data

In [188]:
data.dropna(axis=1, how='all') # drop the columns that are all NA

In [189]:
df = DataFrame(np.random.randn(7, 3))
df

In [190]:
df.iloc[:4, 1] = NA # first 4 rows on each 1st column set to NA
df.iloc[:2, 2] = NA # first 2 rows on each 2nd column set to NA
df

In [191]:
df.dropna()

In [192]:
df.dropna(thresh=2) # keep rows that at least has n not NA value

### Filling In Missing Data

Rather than filtering out missing data, we may want to fill in the "holes" in any number of ways.

In [193]:
df.fillna(0) # fill NA values with 0

In [194]:
df.fillna({1: 0.5, 2: 0}) # using dict, for every key is a column stating that every missing value in each column has a different value

In [195]:
_ = df.fillna(0, inplace=True) # modifying the existing object in place
df

In [196]:
df = DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA # next first 2 elements on each 1st column are set to NA
df.iloc[4:, 2] = NA # next first 4 elements on each 2nd column are set to NA
df

In [197]:
df.fillna(method='ffill') # fills NA values with the last non NA value

In [198]:
df.fillna(method='ffill', limit=1) # fill NaN values only n times for each column

In [199]:
df.fillna(method='ffill', limit=2) # fill NaN values only n times for each column, maximum number of consecutive periods to fill

In [200]:
data = Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean()) # sets NA values to the mean/median of the Series

## Data Transformation

### Removing Duplicates

In [201]:
data = DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [202]:
data.duplicated() # returns True if at least a row has been observed before

In [203]:
data.drop_duplicates() # drops any row that has been seen before

In [204]:
data['v1'] = range(7)
data

In [205]:
data.drop_duplicates(['k1']) # dropping rows on any seen before values inside k1

In [206]:
data.drop_duplicates(['k1', 'k2']) # dropping rows on any seen before values inside k1 and k2

In [207]:
data.drop_duplicates(['k1', 'k2'], keep='last') # dropping first observed rows and keeping the last duplicated rows

### Transforming Data Using a Function or Mapping

In [208]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                           'Pastrami', 'corned beef', 'Bacon',
                           'pastrami', 'honey ham', 'nova lox'],
                           'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [209]:
meat_to_animal = {
      'bacon': 'pig',
      'pulled pork': 'pig',
      'pastrami': 'cow',
      'corned beef': 'cow',
      'honey ham': 'pig',
      'nova lox': 'salmon'
}

In [210]:
lowercased = data['food'].str.lower()
lowercased

In [211]:
data['animal'] = lowercased.map(meat_to_animal) # mapping dictionary key value to df
data

In [212]:
data['food'].map(lambda x: meat_to_animal[x.lower()]) # alternative way to lowercasing

### Replacing Values

In [213]:
data = Series([1., -999., 2., -999., -1000., 3.])
data

In [214]:
data.replace(-999, np.nan) # replace -999 with NA

In [215]:
data.replace([-999, -1000], np.nan) # replace multiple values

In [216]:
data.replace([-999, -1000], [np.nan, 0]) # replace -999 with NA and -1000 with 0

In [217]:
data.replace({-999: np.nan, -1000: 0}) # same like above

### Renaming Axis Indexes

In [218]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

In [219]:
transform = lambda x: x[:4].upper()
data.index = data.index.map(transform) # maps indexes with transform
data

In [220]:
data.rename(index=str.title, columns=str.upper) # modifying index and column

In [221]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

In [222]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretization and Binning

In [223]:
ages = [1, 20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins) # groups ages from 18-25, 25-35, 46-60, 61-100
cats

In [224]:
cats.codes # -1 indicates that does not belong in any group

In [225]:
cats.categories

In [226]:
pd.value_counts(cats)

In [227]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False) # excluding right most value (ex. 18-25)

In [228]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names) # each categories are labeled sequentially according to group_names

In [229]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2).categories # precision=2 -> 2 decimal point
# this creates 4 categories and all differences of are equal from each left or right categories

In [230]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4) # cut into quartiles
cats

In [231]:
pd.value_counts(cats) # quartiles make sure that each categories have the same value_counts

In [232]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

In [233]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

### Detecting and Filtering Outliers

In [234]:
data = DataFrame(np.random.randn(1000, 4))
data.describe()

In [235]:
col = data[2]
col[np.abs(col) > 3] # get values > 3 (absolute) on column 2

In [236]:
data[(np.abs(data) > 3).any(True)] # get any rows that have a 3 or -3 value in its row

In [237]:
data[np.abs(data) > 3] = np.sign(data) * 3 # np.sign returns -1 if x < 0, 0 if x==0, 1 if x > 0
data.describe()

In [238]:
np.sign(data).head()

### Permutation and Random Sampling

In [239]:
df = DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

In [240]:
df

In [241]:
df.take(sampler) # move indexes according to sampler's values

In [242]:
df.sample(3) # taking random subset without replacement

In [243]:
choices = Series([5, 7, -1, 6, 4])
draws = choices.sample(10, replace=True) # replace=True to allow repeat choices
draws # takes 10 random samples (this is valid if replace=True), if replace=False then this will result an error

### Computing Indicator/Dummy Variables

In [244]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
df

In [245]:
pd.get_dummies(df['key']) # get unique values of key then converts it into columns containing all 1s and 0s

In [246]:
dummies = pd.get_dummies(df['key'], prefix='key') # prefix='key' -> key_a, key_b, key_c (according to unique values)
df_with_dummy = df[['data1']].join(dummies) # get column of data1 then join with dummies
df_with_dummy

#### Useful way of using `get_dummies`

In [247]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('../input/pandas-data-loading-storage-file-formats/movies.dat', sep='::', header=None, names=mnames)
movies[:10]

In [248]:
all_genres = [y for x in movies.genres for y in x.split('|')] # same like below
# for x in movies.genres:
#     all_genres.extend(x.split('|')) # adds all elements from x.split to all_genres
genres = pd.unique(all_genres)
genres

In [249]:
# one way to construct the dummy dataframe is to start a DF of all zeros
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = DataFrame(zero_matrix, columns=genres)
dummies.head()

In [250]:
gen = movies.genres[0]
gen

In [251]:
gen.split('|')

In [252]:
dummies.columns.get_indexer(gen.split('|')) # get index of columns of gen.split("|") values

In [253]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|')) # just like above
    dummies.iloc[i, indices] = 1 # i refers to the row index, indices refers to columns that we want to mark as 1
dummies.head()

In [254]:
movies_windic = movies.join(dummies.add_prefix('Genre_')) # join movies with dummies
movies_windic.iloc[0] # first row

## String Manipulation

### String Object Methods

In [255]:
val = 'a,b,  guido'
val.split(',')

In [256]:
pieces = [x.strip() for x in val.split(',')]
pieces

In [257]:
first, second, third = pieces
first + '::' + second + '::' + third

In [258]:
'::'.join(pieces) # faster than above

In [259]:
'guido' in val, val.index(','), val.find(':')

In [260]:
val.count(',') # count occurrences

In [261]:
val.replace(',', '::')

In [262]:
val.replace(',', '')

### Regular Expressions

In [263]:
import re
text = "foo bar\t baz \tqux"

In [264]:
re.split('\s+', text) # \s+ -> one or more whitespace characters

In [265]:
regex = re.compile('\s+') # compiled the regex (much more efficient as it save CPU cycles especially when using the same pattern over and over)
regex.split(text) # apply to text (just like above but with more deeper detail)

In [266]:
regex.findall(text) # get list of all patterns matching the regex

In [267]:
text = """Dave dave@google.com
    Steve steve@gmail.com
    Rob rob@gmail.com
    Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}' # finding email email addresses
regex = re.compile(pattern, flags=re.IGNORECASE) # makes the regex case-insensitive

In [268]:
regex.findall(text)

In [269]:
m = regex.search(text) # finds the first matching regex
m

In [270]:
text[m.start():m.end()]

In [271]:
print(regex.match(text)) # match => if the patterns occurs at the start of the string

In [272]:
print(regex.sub('REDACTED', text)) # replace any value with 'REDACTED' if matches regex

In [273]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})' # finds username, domain name and domain suffix from an email address
regex = re.compile(pattern, flags=re.IGNORECASE)
m = regex.match('roderik@gmail.com')
m.groups()

In [274]:
regex.findall(text)

In [275]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text)) # \1 first matched group and so on

### Vectorized String Functions in pandas

In [276]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Roderik': np.nan}
data = Series(data)
data

In [277]:
data.isnull()

In [278]:
data.str.contains('gmail')

In [279]:
data.str.findall(pattern, flags=re.IGNORECASE)

In [280]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches